In [4]:
import os
import numpy as np
from PIL import Image
import json
import tensorflow as tf
import cv2

In [5]:
img_path = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/원천데이터/TS_damage'

label_path = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/라벨링데이터/TL_damage'

In [9]:
def get_data(path):
                                                         
    def make_image_dataset(img_name):
        image_path = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/원천데이터/TS_damage/' + img_name +'.jpg'
        image = Image.open(image_path)
        image = image.resize((512, 512))
        image = np.array(image)
        return image
    
    def make_segmentation_map_dataset(json_file):
        json_path = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/라벨링데이터/TL_damage/' + json_file + '.json'
        with open(json_path, 'r', encoding='UTF-8') as json_file:
            json_data = json.load(json_file)

        img = np.zeros((json_data['images']['height'], json_data['images']['width'], 3), np.uint8)

        for ann in json_data['annotations']:
            if 'segmentation' in ann:
                segmentation = ann['segmentation'][0][0]
                cv2.fillPoly(img, [np.array(segmentation)], (255, 255, 255))
        img = np.resize(img, (512,512,))
        return img
    
    
    def generator():
        for filename in os.listdir(path):
            final_img = make_image_dataset(filename.split('.')[0])
            final_map = make_segmentation_map_dataset(filename.split('.')[0])
            yield (final_img, final_map)
    

    
    dataset =  tf.data.Dataset.from_generator(generator,
                                              (tf.int64, tf.int64))
    dataset = dataset.batch(BATCH_SIZE)
    
    return dataset

In [10]:
BATCH_SIZE = 16
datasets = get_data(img_path)

In [11]:
for final_img, final_map in iter(datasets.take(1)):
    print(final_img)

tf.Tensor(
[[[[ 53  63  55]
   [ 51  61  53]
   [ 52  59  52]
   ...
   [117  69  72]
   [135  82  85]
   [126  70  73]]

  [[ 51  61  53]
   [ 51  61  53]
   [ 54  61  54]
   ...
   [123  75  78]
   [139  86  90]
   [124  69  73]]

  [[ 49  59  51]
   [ 51  61  53]
   [ 55  62  55]
   ...
   [121  73  76]
   [116  65  69]
   [124  69  74]]

  ...

  [[190 203 178]
   [191 199 176]
   [186 190 169]
   ...
   [ 58  48  45]
   [ 53  41  39]
   [ 47  32  32]]

  [[191 201 177]
   [187 193 171]
   [186 187 166]
   ...
   [ 58  48  45]
   [ 56  44  42]
   [ 65  51  51]]

  [[194 202 179]
   [185 190 168]
   [185 185 165]
   ...
   [ 60  50  47]
   [ 72  60  58]
   [ 63  49  49]]]


 [[[189 173 173]
   [187 172 172]
   [187 174 172]
   ...
   [197 197 197]
   [197 197 197]
   [197 197 197]]

  [[179 163 163]
   [181 167 166]
   [181 168 167]
   ...
   [197 197 197]
   [197 197 197]
   [197 197 197]]

  [[166 153 151]
   [172 161 159]
   [173 163 161]
   ...
   [197 197 197]
   [197 197 197]
